In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils.extmath import safe_sparse_dot

import numpy as np
import pandas as pd
import math

In [3]:
# Tokyo Race Data (Import data)
# race_data = pd.read_csv("./Downloads/Dataset 0-2.csv",  
#                   names=["Distance", "Altitude"])

# #Flanders Men's Data
# flanders_men_data = pd.read_csv("./Downloads/Flanders Men_s.csv",  
#                   names=["Distance", "Altitude"])

# #Flanders Women's Data
# flanders_women_data = pd.read_csv("./Downloads/Flanders_ Women.csv",  
#                   names=["Distance", "Altitude"])


In [8]:
# Define constants
A_m = 1.9 # male surface area
A_f = 1.6 # female surface area
M_m = 88.9 # male mass (kg)
M_f = 71.22 # female mass (kg)
K_a = 0.0108 # air resistance
K_r = 0.143 # rolling resistance
g = 0.00981 # gravitational constant
R = 0.0063 # recovery (s^-1)
MVC = 1000 # watt/newton
k = 0.0153 # fatigue (s^-1)
v_w = 1 # velocity wind
theta_w = 1 # wind direction
mu = 0.75 # coefficient of friction
P_eq = MVC * (R/(2*k))*(-1+(math.sqrt(1+4*(k/R))))

# To populate
theta_rider = [0,20,20,0,20,20,0,20,20,0]
gradient_rider = [-0.2,0.5,0.8,-0.2,0.5,0.8,-0.2,0.5,0.8,-0.2]
force_arr = [10,20,30,40,50]
r_rider = [50,4000,200,50,4000,200,50,4000,200,50]
segment_len_arr = [34,53,12,34,53,12,34,53,12,33] 

In [103]:
# Male

# Quadratic solver to get new velocity given a particular applied force
def get_new_velocity(mass, prev_vel, force, i):
#     a = 0 - (mass/2)
#     b = (1 / prev_vel) + (mass * prev_vel)
#     c = (K_a * A_m * prev_segment_len * pow(prev_vel + (v_w * math.sin(abs(theta_rider[i-1] - theta_w))), 2) 
#     + (g * gradient_rider[i-1] * mass * prev_segment_len) + (K_r * mass * prev_segment_len) 
#     + ((mass * pow(prev_vel,2)) / 2)) / (force * prev_segment_len)
    
    prev_segment_len = segment_len_arr[i-2] if i > 1 else segment_len_arr[0]
    a = (1 / 2) * (mass * prev_vel / prev_segment_len)
    b = 0 - force
    c = (K_a * A_m * prev_vel * pow(prev_vel + (v_w * math.sin((math.pi / 2) - abs(theta_rider[i-1] - theta_w))), 2) 
    - (g * gradient_rider[i-1] * mass * prev_vel) + (K_r * mass * prev_vel) 
    - ((mass * pow(prev_vel,3)) / (2 * prev_segment_len)))
       
    d = (b**2 - 4*a*c)
    if d < 0: d = 0
    result = (-b + math.sqrt(d)) / (2 * a)
    return result

# Call first instance of recursion
def dfs():
    final_max = -10000000

    to_return = []
    for j in range(5):
        optimal_arr = [force_arr[j]]
        print("here",len(optimal_arr))
        
        gr = dfs_helper(force_arr[j], 1000, 10, 0, 0, 0, 1, optimal_arr)
        d= gr[0]
        op = gr[1]
        
        
        if d > final_max:
            final_max = d
            print("yuh",len(optimal_arr)) # After testing first force (all the way down the tree and back up, optimal arr has length 1673!)
            if len(optimal_arr) == len(segment_len_arr):
                optimal_arr.pop(len(optimal_arr)-1)
            #print(j, d, len(to_return))
            to_return = optimal_arr
            
    return final_max, to_return
    
def dfs_helper(force, prev_f_max, prev_vel, prev_vel_avg, prev_force, r, i, arr):
    op = []
    #print(len(arr))
    if i == len(segment_len_arr):
        maxi = -10000000
        # At last race segment, no more recursive calls. Just return the fastest possible average velocity
        for j in range(5):
            temp_arr = arr
            new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
            new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
         
            if (check_constraints(force, new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
                if (((prev_vel_avg * (i-1)) + new_vel) / i) > maxi:
                    maxi = (((prev_vel_avg * (i-1)) + new_vel) / i)        
                    temp_arr.append(force_arr[j])
                    op = temp_arr
                    
        return (maxi, op)
    
    new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
    max_v_avg = -10000000
    to_return_arr = []
    for j in range(5):
        print(len(arr))
        temp_arr = arr
        # Calculate new velocity given chosen force
        new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
        
        # If the new velocity and max force are within constraints, run DFS, if not, then go to next iteration.
        if (check_constraints(force_arr[j], new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
            new_vel_avg = new_vel if (i == 0) else ((prev_vel_avg * (i-1)) + new_vel) / i
            
            # Recursive call here
            temp_arr.append(force_arr[j])
            rec_res = dfs_helper(force_arr[j], new_f_max, new_vel, new_vel_avg, force, r_rider[i], i+1, temp_arr)
            yuh = rec_res[0]
            if yuh > max_v_avg: 
                max_v_avg = yuh
                to_return_arr = rec_res[1]
                
       # if not len(temp_arr) == 0: temp_arr.pop(len(temp_arr)-1)
    return (max_v_avg, to_return_arr)

# Check for our 4 constraints
def check_constraints(force, new_f_max, prev_vel, new_vel, prev_segment_len, t):
    sof_bool = (new_f_max >= P_eq) and (new_f_max <= MVC)
    r_prev = r_rider[0] if t < 2 else r_rider[t-2]
    prev_segment_len = segment_len_arr[t-2] if t > 2 else segment_len_arr[1]
    velocity_bool = ((force * math.sqrt(mu * g * r_rider[t-1])) - 
                (K_a * A_m * math.sqrt(mu * g * r_rider[t-1]) * pow((math.sqrt(mu * g * r_rider[t-1]) + v_w * 
                math.sin(abs(theta_rider[t-1] - theta_w))),2) - (g * gradient_rider[t-1] * M_m * 
                math.sqrt(mu * g * r_rider[t-1])) + (K_r *  math.sqrt(mu * g * r_rider[t-1]) * M_m) + 
                 (M_m * pow((math.sqrt(mu * g * r_rider[t-1]) - math.sqrt(mu * g * r_prev)),2) 
                  * math.sqrt(mu * g * r_rider[t-1]) / (2 * prev_segment_len)))) <= 0 # constrain r
    return (new_vel <= 45) and (new_vel >= 0) and sof_bool and velocity_bool

In [104]:
dfs()

here 1
1
2
3
4
5
6
7
8
9
15
16
16
16
16
17
23
24
24
24
24
25
31
32
32
32
32
33
39
40
40
40
40
40
40
40
40
40
41
42
43
49
50
50
50
50
51
57
58
58
58
58
59
65
66
66
66
66
67
73
74
74
74
74
74
74
74
74
74
74
74
74
75
76
77
78
84
85
85
85
85
86
92
93
93
93
93
94
100
101
101
101
101
102
108
109
109
109
109
109
109
109
109
109
110
111
112
118
119
119
119
119
120
126
127
127
127
127
128
134
135
135
135
135
136
142
143
143
143
143
143
143
143
143
143
143
143
143
144
145
146
147
153
154
154
154
154
155
161
162
162
162
162
163
169
170
170
170
170
171
177
178
178
178
178
178
178
178
178
178
179
180
181
187
188
188
188
188
189
195
196
196
196
196
197
203
204
204
204
204
205
211
212
212
212
212
212
212
212
212
212
212
212
212
213
214
215
216
222
223
223
223
223
224
230
231
231
231
231
232
238
239
239
239
239
240
246
247
247
247
247
247
247
247
247
247
248
249
250
256
257
257
257
257
258
264
265
265
265
265
266
272
273
273
273
273
274
280
281
281
281
281
281
281
281
281
281
281
281
281
281
281
281
2

101
101
101
101
102
108
109
109
109
109
109
109
109
109
109
110
111
112
118
119
119
119
119
120
126
127
127
127
127
128
134
135
135
135
135
136
142
143
143
143
143
143
143
143
143
143
143
143
143
144
145
146
147
153
154
154
154
154
155
161
162
162
162
162
163
169
170
170
170
170
171
177
178
178
178
178
178
178
178
178
178
179
180
181
187
188
188
188
188
189
195
196
196
196
196
197
203
204
204
204
204
205
211
212
212
212
212
212
212
212
212
212
212
212
212
213
214
215
216
222
223
223
223
223
224
230
231
231
231
231
232
238
239
239
239
239
240
246
247
247
247
247
247
247
247
247
247
248
249
250
256
257
257
257
257
258
264
265
265
265
265
266
272
273
273
273
273
274
280
281
281
281
281
281
281
281
281
281
281
281
281
281
281
281
281
281
282
283
284
285
286
287
293
294
294
294
294
295
301
302
302
302
302
303
309
310
310
310
310
311
317
318
318
318
318
318
318
318
318
318
319
320
321
327
328
328
328
328
329
335
336
336
336
336
337
343
344
344
344
344
345
351
352
352
352
352
352
352
352
352


1477
1477
1477
1478
1484
1485
1485
1485
1485
1486
1492
1493
1493
1493
1493
1494
1500
1501
1501
1501
1501
1501
1501
1501
1501
1501
1502
1503
1504
1510
1511
1511
1511
1511
1512
1518
1519
1519
1519
1519
1520
1526
1527
1527
1527
1527
1528
1534
1535
1535
1535
1535
1535
1535
1535
1535
1535
1535
1535
1535
1536
1537
1538
1539
1545
1546
1546
1546
1546
1547
1553
1554
1554
1554
1554
1555
1561
1562
1562
1562
1562
1563
1569
1570
1570
1570
1570
1570
1570
1570
1570
1570
1571
1572
1573
1579
1580
1580
1580
1580
1581
1587
1588
1588
1588
1588
1589
1595
1596
1596
1596
1596
1597
1603
1604
1604
1604
1604
1604
1604
1604
1604
1604
1604
1604
1604
1605
1606
1607
1608
1614
1615
1615
1615
1615
1616
1622
1623
1623
1623
1623
1624
1630
1631
1631
1631
1631
1632
1638
1639
1639
1639
1639
1639
1639
1639
1639
1639
1640
1641
1642
1648
1649
1649
1649
1649
1650
1656
1657
1657
1657
1657
1658
1664
1665
1665
1665
1665
1666
1672
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673
1673


(11.520126099231433,
 [10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  40,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  40,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  10,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  40,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  40,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
  10,
  10,
  10,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  20,
  10,
  10,
  20,
  30,
  40,
  50,
  20,
  30,
 